In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch.utils.data as Data
import torch
from torch.nn import Module, LSTM, Linear
from torch.utils.data import DataLoader, TensorDataset
from function import LSTM_Net

C:\Users\86153\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 数据

In [2]:
# read_data = pd.read_csv('../data/date.csv')
# read_data = read_data.set_index('Date')
# read_data = read_data.loc['2014-01-01':]
# # read_data
# fulldata = read_data[[
# #                     'AS51',                         #0.431
# #                  'CKLSE',                       #0.265
# #                  'CSX5P',                   #0.318
# #                  'DJI',                 #0.43
# #                  'FCHI',            #0.17
# #                  'FTSE',        #0.58
# #                  'GDAXI',   #0.391
# #                  'HSI',      #0.374
# #                  'IBOVESPA',   #0.298
#                  'IXIC',         #0.524
# #                  'KS11',           #0.26
# #                  'N225',            #0.621
# #                  'RUT',              #0.6
# #                  'SENSEX',            #0.282
# #                  'SPTSX',               #0.39
# #                  'SPX',                   #0.675
# #                  'TWII',                    #0.3
# #                  'shanghai',                    #0.074
#                  'shenzheng']]

In [3]:
# read_data = pd.read_csv('../data/bond/date.csv')
# read_data = read_data.set_index('Date')
# read_data = read_data.loc['2014-01-01':]
# fulldata = read_data[[
# #                         'Australia',
# #                       'Brazil',
# #                       'Canada',
#                         # 'Germany',
# #                         'France',
# #                       'Hongkong',
# #                       'India',
# #                       'Japan',
#                       'Malaysia',
# #                       'US' ,
#                       'Close']]

In [4]:
read_data = pd.read_csv('../data/rate/date.csv')
read_data = read_data.set_index('Date')
read_data = read_data.loc['2014-01-01':]
fulldata = read_data[[
#                         'Australia',
#                        'Brazil',
#                       'Canada',
#                       'EU',
#                       'Hongkong',
#                       'India', 
#                       'Japan',
#                       'Korea',
#                       'Malaysia',
#                       'Taiwan', 
                      'UK',
#                       'US' ,
                      'Close']]

In [5]:
# a = fulldata[['Close']]
# fulldata = pd.concat([a, a], axis=1)

# 参数设置

In [6]:
predict_day = 1             # 预测未来几天
time_step = 25              # 这个参数很重要，是设置用前多少天的数据来预测，也是LSTM的time step数，请保证训练数据量大于它

In [7]:
# #超参数
# batch_size = 32
# hidden_size = 128           # LSTM的隐藏层大小，也是输出大小
# lstm_layers = 2             # LSTM的堆叠层数
# dropout_rate = 0.001          # dropout概率
# learning_rate = 0.001

In [8]:
c = len(fulldata.columns.tolist())
 # 数据参数
if(c>2):
    feature_columns = list(range(0, c-1))     # 要作为feature的列，按原数据从0开始计算，也可以用list 如 [2,4,6,8] 设置
    label_columns = [c-1]                  # 要预测的列，按原数据从0开始计算, 如同时预测第四，五列 最低价和最高价
else:
    feature_columns = [0]
    label_columns = [1]

shuffle_train_data = True   # 是否对训练数据做shuffle
random_seed = 32            # 随机种子，保证可复现

train_data_rate = 0.80      # 训练数据占总体数据比例，测试数据就是 1-train_data_rate
valid_data_rate = 0.20      # 验证数据占训练数据比例，验证集在训练过程使用，为了做模型和参数选择

# do_continue_train = False    # 每次训练把上一次的final_state作为下一次的init_state，仅用于RNN类型模型，目前仅支持pytorch
do_continue_train = True 
start_num_in_test = 0      # 测试集中前几天的数据会被删掉，因为它不够一个time_step

data, data_column_name = fulldata.values, fulldata.columns.tolist()
data_num = data.shape[0]
train_num = int(data_num * train_data_rate)
std = np.std(data, axis=0)
mean = np.mean(data, axis=0)              # 数据的均值和方差
norm_data = (data - mean)/std   # 归一化，去量纲

In [9]:
# 定义参数范围
parameters = {
    'Batch_size': [32, 64],
    'lstm_layers': [1,2,4],
    'hidden_size': [64,128,256],
    'Learning_rate': [0.001,0.01],
    'dropout_rate': [0.001,0.01]
}

# 生成所有可能的参数组合
parameter_combinations = []
for batch_size in parameters['Batch_size']:
    for hidden_layer in parameters['lstm_layers']:
        for neurons in parameters['hidden_size']:
            for learning_rate in parameters['Learning_rate']:
                for dropout_rate in parameters['dropout_rate']:
                    parameter_combinations.append({
                        'Batch_size': batch_size,
                        'lstm_layers': hidden_layer,
                        'hidden_size': neurons,
                        'Learning_rate': learning_rate,
                        'dropout_rate': dropout_rate
                })

# #打印所有参数组合
# for idx, params in enumerate(parameter_combinations):
#     print(f"参数组合 {idx+1}: {params}")

# 初始化函数

In [10]:
def get_train_and_valid_data():
#     feature_data = norm_data[:train_num, feature_columns]
    feature_data = norm_data[:train_num]
    label_data = norm_data[time_step : predict_day + train_num,
                                    label_columns]    # 将延后几天的数据作为label

    
       # 在非连续训练模式下，每time_step行数据会作为一个样本，两个样本错开一行，比如：1-20行，2-21行。。。。
    train_x = [feature_data[i:i+time_step] for i in range(train_num-time_step)]
    train_y = [label_data[i:i+predict_day] for i in range(train_num-time_step)]

    train_x, train_y = np.array(train_x), np.array(train_y)

    train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=valid_data_rate,
                                                              random_state=random_seed,
                                                              shuffle=shuffle_train_data)   # 划分训练和验证集，并打乱
    
    train_X, valid_X, train_Y, valid_Y = train_x, valid_x, train_y, valid_y
    train_and_valid_data=[train_X, train_Y, valid_X, valid_Y]

    train_X, train_Y, valid_X, valid_Y = train_and_valid_data
    train_X, train_Y = torch.from_numpy(train_X).float(), torch.from_numpy(train_Y).float()     # 先转为Tensor
    train_loader = DataLoader(TensorDataset(train_X, train_Y), batch_size=batch_size)    # DataLoader可自动生成可训练的batch数据

    valid_X, valid_Y = torch.from_numpy(valid_X).float(), torch.from_numpy(valid_Y).float()
    valid_loader = DataLoader(TensorDataset(valid_X, valid_Y), batch_size=batch_size)    
    
    
    
    
    return train_loader, valid_loader

In [11]:
def restore_data(pred_result, test_Y):
    # 假设这是数据还原的函数
    # 在这里实现将归一化后的数据还原成原始数据的过程
    # 返回还原后的数据
    std1 = std[1]
    mean1 = mean[1]
    true = test_Y*std1 + mean1
    pred = pred_result*std1 + mean1
    return pred, true


def train(model, dataloader, optimizer):
    train_loss_array = []
    train_loss = []
    for i, _data in enumerate(train_loader):
        _train_X, _train_Y = _data[0],_data[1]
        hidden_train = None  # 手动创建新的隐藏状态
        optimizer.zero_grad()               # 训练前要将梯度信息置 0
        pred_Y, hidden_train = model(_train_X, hidden_train)    # 这里走的就是前向计算forward函数
        h_0, c_0 = hidden_train
        h_0.detach_(), c_0.detach_()    # 去掉梯度信息
        hidden_train = (h_0, c_0)
        # 假设你的目标张量为 target
        _train_Y = _train_Y.squeeze(dim=2)
        pred_Y = pred_Y[:,-1]
        _train_Y = _train_Y[:,-1]
        
        loss = criterion(pred_Y, _train_Y)  # 计算loss
        loss.backward()                     # 将loss反向传播
        optimizer.step()                    # 用优化器更新参数
        train_loss_array.append(loss.item())
    train_loss.append(np.mean(train_loss_array))
    train_loss_cur = np.mean(train_loss_array)
    return train_loss_cur

def eval(model, dataloader):
  # 先定义一个tensor保存预测结果
    result_valid = torch.Tensor()
    true_valid = torch.Tensor()
    model.eval()                    # pytorch中，预测时要转换成预测模式
    valid_loss_array = []
    hidden_valid = None
    val_loss = []
    
    for _valid_X, _valid_Y in valid_loader:
        # 使用模型的初始隐藏状态
        hidden_valid = None 
        pred_Y, hidden_valid = model(_valid_X, hidden_valid)
        _valid_Y = _valid_Y.squeeze(dim=2)
        pred_Y = pred_Y[:,-1]
        _valid_Y = _valid_Y[:,-1]
        loss = criterion(pred_Y, _valid_Y)  # 验证过程只有前向计算，无反向传播过程
        valid_loss_array.append(loss.item())
        
        cur_pred = torch.squeeze(_valid_Y, dim=0)
        true_valid = torch.cat((true_valid, cur_pred), dim=0)
        
        cur_pred = torch.squeeze(pred_Y, dim=0)
        result_valid = torch.cat((result_valid, cur_pred), dim=0)
    true_valid = true_valid.detach().numpy() 
    result_valid = result_valid.detach().numpy()    
        
    val_loss.append(np.mean(valid_loss_array))
    valid_loss_cur = np.mean(valid_loss_array)
    
    return valid_loss_cur, true_valid, result_valid

In [12]:
 # 网络参数
# input_size = norm_data.shape[1]-1
input_size = norm_data.shape[1]
output_size = predict_day

results = []
COUNT = len(parameter_combinations)


for i in range(len(parameter_combinations)):
# for i in range(3):
        # 假设你想选择第三个参数组合
    selected_params = parameter_combinations[i]

    # 然后你就可以按照键来获取对应的值
    batch_size = selected_params['Batch_size']
    lstm_layers = selected_params['lstm_layers']
    hidden_size = selected_params['hidden_size']
    learning_rate = selected_params['Learning_rate']
    dropout_rate = selected_params['dropout_rate']
    

    
    train_loader, valid_loader = get_train_and_valid_data()
    
    model = LSTM_Net(input_size, hidden_size, output_size, lstm_layers, dropout_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()      # 这两句是定义优化器和loss
    best_loss = 5
    epochs = 100 # 整个训练集被训练多少遍，不考虑早停的前提下
    patience = 40               # 训练多少epoch，验证集没提升就停掉
    bad_epoch = 0
    vl_loss = []
    for epoch_i in range(epochs):
        train_loss = train(model, train_loader, optimizer)

        eval_loss, tru, pre = eval(model, valid_loader)

        vl_loss.append(eval_loss)
        if eval_loss < best_loss:
            best_loss = eval_loss
            best_epoch = epoch_i
            best_result_valid = pre
            best_true_valid = tru
    #         torch.save(model.state_dict(), '../model/best_ANN_trainModel.pth')  # 模型保存
        else:
            bad_epoch += 1
            if bad_epoch >= patience:    # 如果验证集指标连续patience个epoch没有提升，就停掉训练
                break
    results.append({
                    'epoch': best_epoch,
                    'loss': best_loss})
    print(COUNT,i)

D:\Anaconda3\envs\py39\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.001 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


72 0


D:\Anaconda3\envs\py39\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.01 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


72 1


D:\Anaconda3\envs\py39\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.001 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


72 2
72 3
72 4
72 5
72 6
72 7
72 8
72 9
72 10
72 11
72 12
72 13
72 14
72 15
72 16
72 17
72 18
72 19
72 20
72 21
72 22
72 23
72 24
72 25
72 26
72 27
72 28
72 29
72 30
72 31
72 32
72 33
72 34
72 35
72 36
72 37
72 38
72 39
72 40
72 41
72 42
72 43
72 44
72 45
72 46
72 47
72 48
72 49
72 50
72 51
72 52
72 53
72 54
72 55
72 56
72 57
72 58
72 59
72 60
72 61
72 62
72 63
72 64
72 65
72 66
72 67
72 68
72 69
72 70
72 71


In [13]:
for idx, params in enumerate(results):
    print(f"参数组合 {idx+1}: {params}")

参数组合 1: {'epoch': 61, 'loss': 0.007269594779548545}
参数组合 2: {'epoch': 76, 'loss': 0.007409100731213887}
参数组合 3: {'epoch': 9, 'loss': 0.008173541515134275}
参数组合 4: {'epoch': 8, 'loss': 0.007968709610092143}
参数组合 5: {'epoch': 56, 'loss': 0.00759950610032926}
参数组合 6: {'epoch': 48, 'loss': 0.007512958642716209}
参数组合 7: {'epoch': 50, 'loss': 0.008405869904284676}
参数组合 8: {'epoch': 36, 'loss': 0.00899818679317832}
参数组合 9: {'epoch': 89, 'loss': 0.007059343042783439}
参数组合 10: {'epoch': 85, 'loss': 0.0070414597090954585}
参数组合 11: {'epoch': 31, 'loss': 0.007378391611079375}
参数组合 12: {'epoch': 40, 'loss': 0.008650517906062305}
参数组合 13: {'epoch': 66, 'loss': 0.0075106459359327955}
参数组合 14: {'epoch': 28, 'loss': 0.0075140963308513165}
参数组合 15: {'epoch': 38, 'loss': 0.008397321760033568}
参数组合 16: {'epoch': 41, 'loss': 0.008506667916662991}
参数组合 17: {'epoch': 18, 'loss': 0.007978076542106768}
参数组合 18: {'epoch': 19, 'loss': 0.00815808707072089}
参数组合 19: {'epoch': 56, 'loss': 0.007906260415135572}
参数组合

In [14]:
# 合并参数组合和结果到一个字典列表
combined_params = []
for params, result in zip(parameter_combinations, results):
    combined_params.append({**params, **result})

In [15]:
# 根据损失值排序
sorted_params = sorted(combined_params, key=lambda x: x['loss'])

# 取前三个
top_three_params = sorted_params[:10]

print("Top Three Parameters:")
for i, param in enumerate(top_three_params, start=1):
    print(f"Rank {i}: {param}")

Top Three Parameters:
Rank 1: {'Batch_size': 64, 'lstm_layers': 2, 'hidden_size': 64, 'Learning_rate': 0.01, 'dropout_rate': 0.001, 'epoch': 66, 'loss': 0.006932685462137063}
Rank 2: {'Batch_size': 64, 'lstm_layers': 2, 'hidden_size': 256, 'Learning_rate': 0.01, 'dropout_rate': 0.001, 'epoch': 21, 'loss': 0.006951153045520186}
Rank 3: {'Batch_size': 32, 'lstm_layers': 1, 'hidden_size': 256, 'Learning_rate': 0.001, 'dropout_rate': 0.01, 'epoch': 85, 'loss': 0.0070414597090954585}
Rank 4: {'Batch_size': 32, 'lstm_layers': 1, 'hidden_size': 256, 'Learning_rate': 0.001, 'dropout_rate': 0.001, 'epoch': 89, 'loss': 0.007059343042783439}
Rank 5: {'Batch_size': 64, 'lstm_layers': 1, 'hidden_size': 128, 'Learning_rate': 0.01, 'dropout_rate': 0.01, 'epoch': 39, 'loss': 0.007075757409135501}
Rank 6: {'Batch_size': 64, 'lstm_layers': 1, 'hidden_size': 64, 'Learning_rate': 0.01, 'dropout_rate': 0.01, 'epoch': 82, 'loss': 0.007091796801735957}
Rank 7: {'Batch_size': 64, 'lstm_layers': 2, 'hidden_siz